In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [6]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [7]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [8]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X=df[feats].values
  y=df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [13]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [14]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.47223572384038, 4.328288468270897)

In [15]:
df['brand_cat'] = df['brand'].map(lambda x:str(x).lower()).factorize()[0]
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [16]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [17]:
test = {'key':'value'}
test['key']
str(test)

"{'key': 'value'}"

In [19]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [21]:
literal_eval(str_dict)[0]['value']

['Men']

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [28]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [29]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )

In [30]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_age gender group', 'feat_lens type', 'feat_style code',
       'feat_elastic banded waist', 'feat_size',
       'feat_number of batteries included', 'feat_hammer loop',
       'feat_diameter', 'feat_mpn#', 'feat_recommended location'],
      dtype='object', length=526)

In [31]:
df[df['feat_athlete'].isnull()].shape

(18272, 526)

In [0]:
keys_stat={}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [36]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())

In [40]:
df[df['brand']!=df['feat_brand']][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [41]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'],model)

(-57.3007548779859, 4.1711822338688425)

In [43]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_manufacturer part_cat', 'feat_material_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.13952249899014, 4.272624678548283)

In [51]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2605 ± 0.0099,brand_cat
0.1070 ± 0.0093,feat_material_cat
0.0452 ± 0.0026,feat_gender_cat
0.0207 ± 0.0011,feat_brand_cat
0.0064 ± 0.0007,feat_style_cat


In [50]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.14653036044393, 4.2370661606122555)

In [46]:
df.brand.value_counts(normalize=True)

nike                  0.097210
puma                  0.033315
ralph lauren          0.028775
vans                  0.021116
new balance           0.020295
                        ...   
handolederco.         0.000055
andanines             0.000055
rockport works        0.000055
the felt store        0.000055
maelstrom tactical    0.000055
Name: brand, Length: 1732, dtype: float64

In [47]:
df[df['brand']=='nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [52]:
feats = ['brand_cat','feat_brand_cat','feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats,model)

(-57.04984129138419, 4.284840113781236)

In [58]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [0]:
feats_cat = [x for x in df.columns if 'cat' in x]

In [0]:
feats = ['brand_cat','feat_metal type_cat','feat_shape_cat','feat_brand_cat','feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
#feats += feats_cat
#feats = list(set(feats))
model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats,model)

In [61]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)
print(result)
perm = PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

(-57.31409773440688, 4.240504622301265)


Weight,Feature
0.2557 ± 0.0080,brand_cat
0.1032 ± 0.0075,feat_material_cat
0.0252 ± 0.0026,feat_gender_cat
0.0175 ± 0.0015,feat_brand_cat
0.0127 ± 0.0011,feat_shape_cat
0.0091 ± 0.0015,feat_metal type_cat
0.0033 ± 0.0012,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [62]:
pwd

'/content/drive/My Drive/Colab Notebooks/dw_matrix'

In [0]:
!git add matrix_one/day5.ip